In [3]:
import psycopg2

In [8]:
from configparser import ConfigParser

def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [9]:
import psycopg2

def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')

In [10]:
connect()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


In [43]:
class projek_sql:
    def __init__(self):
        pass 
    
    def open_connect(self):
        params = config()
        self.connection = psycopg2.connect(**params)
        print('Koneksi dibuka')
    
    def query_sql(self, query, kolom):
        self.cursor = self.connection.cursor()       
        print('PostgreSQL database version:')
        self.cursor.execute('SELECT version()')
        db_version = self.cursor.fetchone()
        print(db_version)
        self.cursor.execute(query)
        hasil = self.cursor.fetchall()
        hasil =pd.DataFrame(hasil,columns=kolom)
        return hasil
        
    def close_connect(self):
        self.cursor.close()
        self.connection.close()
        print("Koneksi ditutup")    

In [44]:
import pandas as pd
#test
test = projek_sql()
test.open_connect()
query = '''select * from actor'''
kolom = ['aktor_id', 'first_name', 'last_name','last_update']
df = test.query_sql(query, kolom)

Koneksi dibuka
PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)


In [45]:
df

,aktor_id,first_name,last_name,last_update
0,1,Penelope,Guiness,2013-05-26 14:47:57.620
1,2,Nick,Wahlberg,2013-05-26 14:47:57.620
2,3,Ed,Chase,2013-05-26 14:47:57.620
3,4,Jennifer,Davis,2013-05-26 14:47:57.620
4,5,Johnny,Lollobrigida,2013-05-26 14:47:57.620
...,...,...,...,...
195,196,Bela,Walken,2013-05-26 14:47:57.620
196,197,Reese,West,2013-05-26 14:47:57.620
197,198,Mary,Keitel,2013-05-26 14:47:57.620
198,199,Julia,Fawcett,2013-05-26 14:47:57.620


In [46]:
#1
query1 = '''select a.category_id, a.name, count(b.film_id)
from category a full outer join film_category b
on a.category_id = b.category_id 
where a.name ='Classics'
group by a.category_id'''
test = projek_sql()
test.open_connect()
kolom1 = ['kategori_id', 'kategori', 'num_films']
df1 = test.query_sql(query1, kolom1)
print(df1)

Koneksi dibuka
PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
   kategori_id  kategori  num_films
0            4  Classics         57


In [47]:
#2
query2 = '''select count(title) 
from film where rating='R' and replacement_cost between 5 and 15  
group by rating'''
kolom2 = ['jumlah film']
df2 = test.query_sql(query2, kolom2)
print(df2)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
   jumlah film
0           52


In [48]:
#3
query3='''select a.staff_id, count(a.payment_id), sum(a.amount) 
from payment a full outer join staff b on a.staff_id = b.staff_id 
group by(a.staff_id)'''
kolom3 = ['staff id', 'banyaknya transaksi', 'total pembayaran' ]
df3 = test.query_sql(query3, kolom3)
print(df3)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
   staff id  banyaknya transaksi total pembayaran
0         1                 7292         30252.12
1         2                 7304         31059.92


In [49]:
#4
query4='''select rating, avg(replacement_cost)::numeric(10,2) 
from film group by rating'''
kolom4 = ['rating', 'average rating']
df4 = test.query_sql(query4, kolom4)
print(df4)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
  rating average rating
0      R          20.23
1  NC-17          20.14
2      G          20.12
3     PG          18.96
4  PG-13          20.40


In [51]:
#5
query5='''select first_name || ' ' || last_name AS name, email, sum(amount) 
from customer a full outer join payment b on a.customer_id = b.customer_id 
group by name, email order by sum(amount) desc limit 5'''
kolom5 = ['name', 'email', 'total payment']
df5 = test.query_sql(query5, kolom5)
print(df5)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
             name                              email total payment
0    Eleanor Hunt    eleanor.hunt@sakilacustomer.org        211.55
1       Karl Seal       karl.seal@sakilacustomer.org        208.58
2   Marion Snyder   marion.snyder@sakilacustomer.org        194.61
3  Rhonda Kennedy  rhonda.kennedy@sakilacustomer.org        191.62
4      Clara Shaw      clara.shaw@sakilacustomer.org        189.60


In [53]:
query6='''select a.name,  count(c.customer_id) as jumlah_customer, avg(rental_rate) ::numeric(10,2)
from rental c 
inner join inventory d on c.inventory_id = d.inventory_id 
inner join film_category b on b.film_id = d.film_id 
inner join category a on b.category_id = a.category_id
inner join film e on b.film_id=e.film_id
group by a.name'''
kolom6 = ['Kategori', 'Jumlah Customer yang meminjam', 'average_rating_rate']
df6 = test.query_sql(query6, kolom6)
print(df6)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
       Kategori  Jumlah Customer yang meminjam average_rating_rate
0        Family                           1096                2.70
1         Games                            969                3.13
2     Animation                           1166                2.76
3      Classics                            939                2.64
4   Documentary                           1050                2.62
5           New                            940                3.09
6        Sports                           1179                3.07
7      Children                            945                2.69
8         Music                            830                3.06
9        Travel                            837                3.32
10      Foreign                           1033                2.95
11        Drama                           1060                3.18
12       Horror                     

In [54]:
#7
query7= '''select first_name || ' ' || last_name AS name, email, count(rental_id) 
from customer a full outer join payment b on a.customer_id = b.customer_id 
group by name, email
HAVING count(rental_id) >= 40'''
kolom7 = ['name', 'email', 'jumlah peminjaman']
df7 = test.query_sql(query7, kolom7)
print(df7)

PostgreSQL database version:
('PostgreSQL 13.1, compiled by Visual C++ build 1914, 64-bit',)
           name                            email  jumlah peminjaman
0    Clara Shaw    clara.shaw@sakilacustomer.org                 40
1     Karl Seal     karl.seal@sakilacustomer.org                 42
2  Eleanor Hunt  eleanor.hunt@sakilacustomer.org                 45
